# Imports

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# Constants

In [2]:
torch.manual_seed(1337)
training_style = "scale"
stop_token = ";"
if training_style == "scale":
    block_size = 64
    batch_size = 512
    max_iters = 10000
    eval_interval = 500
    learning_rate = 5e-4
    eval_iters = 200
    n_embd = 384
    n_layer = 6 # Number of block layers
    n_head = 6
    dropout = 0.2
else:
    block_size = 64
    batch_size = 256
    split_pct = 0.9
    max_iters = 5000
    eval_interval = 500
    learning_rate = 1e-3
    eval_iters = 200
    n_embd = 64
    n_layer = 2 # Number of block layers
    n_head = 2
    dropout = 0.2

In [3]:
# Set GPU device
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'
print(device)

cuda


# Dataloader

In [4]:
chars = [';', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '+', '-', '.', '*', '/', '=', 'e', 'i', 'n', 'f']
vocab_size = len(chars)
print("Number of tokens: {}".format(vocab_size))

Number of tokens: 21


In [5]:
encoder = { ch:i for i,ch in enumerate(chars) }
decoder = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [encoder[c] if c in chars else print (s) for c in s]
decode = lambda l: ''.join([decoder[i] for i in l])

In [6]:
def generate_eqn(val1, val2, split_ans=False):
    # Randomly generate operation
    op_idx = torch.randint(0, 4, (1,))
    if op_idx == 0:
        op = "+"
        result = val1 + val2
    elif op_idx == 1:
        op = "-"
        result = val1 - val2
    elif op_idx == 2:
        op = "*"
        result = val1 * val2
    elif op_idx == 3:
        op = "/"
        result = val1 / val2
    if not split_ans:
        return "{}{}{}={}{}".format(val1, op, val2, result, stop_token)
    else:
        return "{}{}{}=".format(val1, op, val2), result

def get_batch(batch_size):
    numbers = torch.randint(-1000000, 1000000, (batch_size+4, 2))
    data = ""
    ix = []
    for idx in range(numbers.size(0) - 1):
        eqn = generate_eqn(numbers[idx, 0], numbers[idx, 1])
        if idx > 2 and idx < (numbers.size()[0]):
            ix.append(torch.randint(len(data) + eqn.index("=") + 1, len(data) + len(eqn), (1,)).item())
        data = data + eqn
    data = torch.tensor(encode(data))
    x = torch.stack([data[i-block_size:i] for i in ix])
    # y here is the target - the next token for any given point in x
    y = torch.stack([data[i-block_size+1:i+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [7]:
x, y = get_batch(batch_size)
for ix in range(0, len(x), 50):
    print(decode(x[ix].tolist()))
    print(decode(y[ix].tolist()))

2-57000=485492;-455705*-486247=221585189135;-248985+906010=65702
-57000=485492;-455705*-486247=221585189135;-248985+906010=657025
97+-539844=-590441;-409465+-27011=-436476;753748--310474=1064222
7+-539844=-590441;-409465+-27011=-436476;753748--310474=1064222;
1=-38748029753;-652318/740049=-0.8814524412155151;-5702+-303186=
=-38748029753;-652318/740049=-0.8814524412155151;-5702+-303186=-
246094;-655253/-208773=3.1385908126831055;-905467+-215159=-11206
46094;-655253/-208773=3.1385908126831055;-905467+-215159=-112062
3808653354644775;-834593*-291975=243680291175;-531598-179764=-71
808653354644775;-834593*-291975=243680291175;-531598-179764=-711
;-726613--459583=-267030;458505+-803804=-345299;650051-753387=-1
-726613--459583=-267030;458505+-803804=-345299;650051-753387=-10
36549=-1.1241774559020996;-159235+-2308=-161543;-616091+-695334=
6549=-1.1241774559020996;-159235+-2308=-161543;-616091+-695334=-
3=1.9169648885726929;596601-521210=75391;-161372+-695393=-856765
=1.9169648885726929;59660

# Model

In [8]:
# Obtains average loss over multiple batches
@torch.no_grad()
def estimate_loss():
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, T = get_batch(batch_size)
            logits, loss = m(X, T)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

In [9]:
class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads):
        super().__init__()
        self.num_heads = num_heads
        self.c_attn = nn.Linear(n_embd, 3 * n_embd, bias=False) # (C, 3C)
        self.register_buffer('tril', torch.tril(torch.ones(1, 1, block_size, block_size)))
        self.dropout1 = nn.Dropout(dropout)
        self.proj = nn.Linear(n_embd, n_embd) # (C, C)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, x):
        # x has shape (B,T,C) - because of initial embeddings
        B, T, C = x.shape

        q, k, v = self.c_attn(x).split(n_embd, dim=2) # (B, T, C) @ (C, 3C) --> (B, T, 3C) splits -> (B, T, C)
        k = k.view(B, T, self.num_heads, C // self.num_heads).transpose(1,2) # (B, nh, T, hs)
        q = q.view(B, T, self.num_heads, C // self.num_heads).transpose(1,2) # (B, nh, T, hs)
        v = v.view(B, T, self.num_heads, C // self.num_heads).transpose(1,2) # (B, nh, T, hs)

        # affinities
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, nh, T, hs) @ (B, nh, hs, T) ---> (B,nh,T,T)
        wei = wei.masked_fill(self.tril[:, :, :T, :T] == 0, float('-inf')) # (B, nh, T, T)
        wei = F.softmax(wei, dim=-1) # (B, nh, T, T)
        wei = self.dropout1(wei) # (B, nh, T, T)
        out = wei @ v # (B, nh, T, T) @ (B, nh, T, hs) --> (B, nh, T, hs)
        out = out.transpose(1, 2).contiguous().view(B,T,C) # (B, T, nh, hs) -> (B, T, C)
        out = self.dropout2(self.proj(out)) # (B, T, C) @ (C, C) --> (B, T, C)
        return out

In [10]:
class FeedForward(nn.Module):

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd), # Projection layer
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [11]:
class Block(nn.Module):

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        self.sa = MultiHeadAttention(n_head)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [12]:
class GptModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # Final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tkn_emb = self.token_embedding_table(idx) # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
        x = tkn_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B, T, C)
        logits = self.lm_head(x) # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)#, ignore_index=encode("\\")[0])
            
        return logits, loss

    def generate(self, idx):
        stop_idx = encode(stop_token)[0]
        # idx is (B, T) array of indices in the current context
        while idx[0, -1].item() != stop_idx:
            # crop idx to last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get predictions
            logits, loss = self(idx_cond)
            # focus only on last time step
            logits = logits[:, -1, :]
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from distribution
            idx_next = torch.argmax(probs, keepdim=True) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [13]:
m = GptModel()
m.to(device)

GptModel(
  (token_embedding_table): Embedding(21, 384)
  (position_embedding_table): Embedding(64, 384)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (c_attn): Linear(in_features=384, out_features=1152, bias=False)
        (dropout1): Dropout(p=0.2, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (dropout2): Dropout(p=0.2, inplace=False)
      )
      (ffwd): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=384, out_features=1536, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1536, out_features=384, bias=True)
          (3): Dropout(p=0.2, inplace=False)
        )
      )
      (ln1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
    )
    (1): Block(
      (sa): MultiHeadAttention(
        (c_attn): Linear(in_features=384, out_features=1152, bias=False)
        (dropout1): Dropout(p=0.2, 

In [14]:
# Number of parameters in model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

10.681365 M parameters


# Training

In [15]:
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

In [16]:
for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch(batch_size)
    
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 3.1138, val loss 3.1137
step 500: train loss 2.0597, val loss 2.0595
step 1000: train loss 2.0137, val loss 2.0139
step 1500: train loss 1.9839, val loss 1.9843
step 2000: train loss 1.9639, val loss 1.9637
step 2500: train loss 1.9506, val loss 1.9506
step 3000: train loss 1.9320, val loss 1.9321
step 3500: train loss 1.9242, val loss 1.9246
step 4000: train loss 1.9073, val loss 1.9078
step 4500: train loss 1.8992, val loss 1.8995
step 5000: train loss 1.8927, val loss 1.8930
step 5500: train loss 1.8899, val loss 1.8893
step 6000: train loss 1.8878, val loss 1.8877
step 6500: train loss 1.8860, val loss 1.8859
step 7000: train loss 1.8829, val loss 1.8830
step 7500: train loss 1.8800, val loss 1.8792
step 8000: train loss 1.8821, val loss 1.8816
step 8500: train loss 1.8807, val loss 1.8801
step 9000: train loss 1.8746, val loss 1.8748
step 9500: train loss 1.8732, val loss 1.8727
step 9999: train loss 1.8728, val loss 1.8734


# Inference

In [31]:
def generate_context():
    # Randomly generate some context as starting point
    numbers = torch.randint(-1000000, 1000000, (10, 2))
    data = encode(stop_token.join([generate_eqn(numbers[ix, 0], numbers[ix, 1]) for ix in range(10)]))
    return data

In [32]:
def generate_rand_eqn():
    # Generate two random numbers & generate random equation
    eqn = torch.randint(-1000000, 1000000, (2, ))
    to_solve, answer = generate_eqn(eqn[0], eqn[1], split_ans=True)
    return to_solve, answer

In [39]:
def random_inference():
    data = generate_context()
    to_solve, answer = generate_rand_eqn()
    # Append eqn to data and send to model
    data = torch.tensor(data + encode(to_solve)).unsqueeze(0).to(device)
    output = decode(m.generate(data)[0].tolist())
    # Print results
    print("Equation: {}".format(output[output.rindex(stop_token, 0, -2)+1:output.rindex("=")+1]))
    print("Prediction: {}".format(output[output.rindex("=")+1:-1]))
    print("Result: {}".format(answer))

In [46]:
random_inference()

Equation: 483338*674665=
Prediction: 324404400830
Result: 326091231770


# Save the model

In [47]:
import time
torch.save(m.state_dict(), './models/calculator_{}.pt'.format(time.strftime("%Y-%m-%d-%H:%M")))